# Important pytroch stuff

inspired from [here](https://spandan-madan.github.io/A-Collection-of-important-tasks-in-pytorch/)

In [1]:
import torch.nn as nn
import torch
from torch.autograd.variable import Variable
from torchvision import datasets, models, transforms

In [4]:
model = models.resnet101(pretrained = False)

## SECTION 1 Freezing the layers of this Resnet for Fine-tuning

- Let us first explore this model's layers and then make a decision as to which ones we want to **freeze**.
- **By freeze we mean that we want the parameters of those layers to be fixed**.
- When fine tuning a model, we are basically taking a **model trained on Dataset A, and then training it on a new Dataset B**. 
- We could potentially start the training from scratch as well, but it would be like re-inventing the wheel. 
- Let me explain why.
    - Suppose, I want to train a dataset to learn to differentiate between a car and a bicycle.
    - Now, I could potentially gather images of both categories and train a network from scratch. 
        - But, given the majority of work already out there, it's easy to find a model trained to identify things like Dogs, cats, and humans. 
        - Admittedly, neither of these 3 look like cars or bicycles. However, it's still better than nothing. 
        - We could start by taking this model, and train it to learn car v/s bicycle. 
- **Gains** : 
    - 1) It will be faster.
    - 2) We need lesser images of cats and bicycles.

(If interested in knowing more, read this - http://cs231n.github.io/transfer-learning/).

Now, let's take a look at the contents of a resnet18. 
- We **use the function .children() for this purpose. This lets us look at the contents/layers of a model**. 
- Then, we **use the .parameters() function to access the parameters/weights of any layer**.
- Finally, every parameter has a property **.requires_grad** which defines whether a parameter is trained or frozen. By default it is True, and the network updates it in every iteration. If it is set to False, then it is not updated and is said to be "frozen".



In [5]:
child_counter = 0
for child in model.children():
    print(" child", child_counter, "is -")
    print(child)
    child_counter += 1

 child 0 is -
Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
 child 1 is -
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 child 2 is -
ReLU(inplace=True)
 child 3 is -
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
 child 4 is -
Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=Fals

Let's saw we want to freeze all parameters up to first BasicBlock of Child 6. First, lets see a parameter and set it to frozen -

In [6]:
for child in model.children():
    for param in child.parameters():
        print("This is what a parameter looks like - \n",param)
        break
    break

This is what a parameter looks like - 
 Parameter containing:
tensor([[[[-2.2695e-02, -5.1941e-03,  2.5564e-03,  ..., -1.6877e-02,
            1.1665e-02, -3.3372e-02],
          [ 8.9671e-03, -2.7296e-02, -2.4369e-02,  ..., -1.3603e-02,
           -7.5877e-03,  1.6829e-02],
          [ 3.0200e-02, -1.3111e-02, -1.0624e-02,  ...,  4.0543e-03,
           -6.7205e-03,  4.3895e-03],
          ...,
          [ 2.1892e-02,  1.7646e-02,  9.4966e-03,  ..., -3.0569e-02,
           -3.6891e-03,  2.0239e-02],
          [-2.9461e-02, -1.3792e-02, -1.2905e-02,  ...,  2.1952e-02,
            2.6613e-02,  2.6484e-02],
          [-3.5574e-03, -4.4521e-03, -2.1964e-02,  ...,  1.9759e-02,
           -1.2040e-03,  1.9527e-02]],

         [[ 6.8018e-04, -3.1781e-03,  1.4091e-02,  ...,  4.4204e-04,
            2.4192e-02, -2.6419e-03],
          [ 8.6764e-03, -1.6171e-03, -1.5436e-02,  ...,  2.6651e-02,
           -2.7381e-02, -1.1339e-02],
          [-2.1712e-02,  4.5484e-02,  3.2553e-02,  ...,  1.7723e-

Evidently, training this will take a lot of calculations. So, by setting a bunch of these to frozen, training becomes much faster. Now, let's freeze up to first BasicBlock of Child 6

In [7]:
child_counter = 0
for child in model.children():
    if child_counter < 6:
        print("child ",child_counter," was frozen")
        for param in child.parameters():
            param.requires_grad = False
    elif child_counter == 6:
        children_of_child_counter = 0
        for children_of_child in child.children():
            if children_of_child_counter < 1:
                for param in children_of_child.parameters():
                    param.requires_grad = False
                print('child ', children_of_child_counter, 'of child',child_counter,' was frozen')
            else:
                print('child ', children_of_child_counter, 'of child',child_counter,' was not frozen')
            children_of_child_counter += 1

    else:
        print("child ",child_counter," was not frozen")
    child_counter += 1

child  0  was frozen
child  1  was frozen
child  2  was frozen
child  3  was frozen
child  4  was frozen
child  5  was frozen
child  0 of child 6  was frozen
child  1 of child 6  was not frozen
child  2 of child 6  was not frozen
child  3 of child 6  was not frozen
child  4 of child 6  was not frozen
child  5 of child 6  was not frozen
child  6 of child 6  was not frozen
child  7 of child 6  was not frozen
child  8 of child 6  was not frozen
child  9 of child 6  was not frozen
child  10 of child 6  was not frozen
child  11 of child 6  was not frozen
child  12 of child 6  was not frozen
child  13 of child 6  was not frozen
child  14 of child 6  was not frozen
child  15 of child 6  was not frozen
child  16 of child 6  was not frozen
child  17 of child 6  was not frozen
child  18 of child 6  was not frozen
child  19 of child 6  was not frozen
child  20 of child 6  was not frozen
child  21 of child 6  was not frozen
child  22 of child 6  was not frozen
child  7  was not frozen
child  8  wa

### Important Note

Now that you have frozen this network, another thing changes to make this work. That is your optimizer. Your optimizer is the one which actually updates these values. By default, the models are written like this -

`optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1)`

But, this will give you an error as this will try to update all the parameters of model. However, you've set a bunch of them to frozen! So, the way to pass only the ones still being updated is -

`optimizer = torch.optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=0.1)`



## SECTION 2 - Model Saving/Loading

There's 2 primary ways in which models are saved in PyTorch. The suggested one is using **"state dictionaries"**. They're faster and requires lower space. Basically, they have no idea of the model structure, they're just the values of the parameters/weights. So, you must create your model with the required architecture and then load the values. The architecture is declared as we did it above.



In [ ]:
# Let's assume we will save/load from a path MODEL_PATH

# Saving a Model
torch.save(model.state_dict(), MODEL_PATH)

# Loading the model.

# First create a model and define it's architecture as done above in this notebook. If you want a custom architecture.
# read below it's been covered below.
checkpoint = torch.load(MODEL_PATH)
model.load_state_dict(checkpoint)

## SECTION 3 - changing last layer, deleting last layer, adding layers

Most people who come to pytorch don't like the fact that they can't do a .pop() to remove last layer. Especially if they've used Keras. So, let's take a look at how these things can be done.

### CHANGING LAST LAYER

In [ ]:
# Load the model
model = models.resnet18(pretrained = False)

# Get number of parameters going in to the last layer. we need this to change the final layer. 
num_final_in = model.fc.in_features

# The final layer of the model is model.fc so we can basically just overwrite it 
#to have the output = number of classes we need. Say, 300 classes.
NUM_CLASSES = 300
model.fc = nn.Linear(num_final_in, NUM_CLASSES)

### DELETING LAST LAYER (OFTEN, WHEN YOU NEED FEATURES OF A LAYER)

In [ ]:
# Load the model
model = models.resnet18(pretrained = False)

We can get the layers by using model.children() as before. Then, we can convert this into a list by using a list() command on it. Then, we can remove the last layer by indexing the list. Finally, we can use the PyTorch function nn.Sequential() to stack this modified list together into a new model. You can edit the list in any way you want. That is, you can delete the last 2 layers if you want the features of an image from the 3rd last layer!

You may even delete layers from the middle of the model. But obviously, this would lead to incorrect number of features going in to the layer after it as most layers change size of image. In this case, you can index that specific layer of the model and overwrite it just as I showed you immediately above!

In [ ]:
new_model = nn.Sequential(*list(model.children())[:-1])

In [ ]:
new_model_2_removed = nn.Sequential(*list(model.children())[:-2])

### Adding layers

Say, you want to add a fully connected layer to the model we have right now. One obvious way would be to edit the list I discussed above and appending to it another layer. However, often times we have such a model trained and want to see if we can load that model, and add just a new layer on top of it. As mentioned above, the loaded model should have the SAME architecture as saved one, and so we can't use the list method.

We need to add layers on top. The way to do this is simple in PyTorch - We just need to create a custom model! And this brings us to our next section - creating custom models!

## SECTION 4 - CUSTOM MODELS : Combining sections 1-3 and adding layers on top

Let's make a custom model. As mentioned above, we will load half of the model from a pre-trained network. This seems complicated, right? Half the model is trained, half is new. Further, we want some of it to be frozen. Some to be update-able. Really, once you've done this, you can do anything with model architectures in PyTorch.

In [ ]:
# Some imports first
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo
import torch
from torch.autograd.variable import Variable
from torchvision import datasets, models, transforms

# New models are defined as classes. Then, when we want to create a model we create an object instantiating this class.
class Resnet_Added_Layers_Half_Frozen(nn.Module):
    def __init__(self,LOAD_VIS_URL=None):
        super(ResnetCombinedFull2, self).__init__()
    
         # Start with half the resnet model, swap out the final layer because that's the model we had defined above. 
        model = models.resnet18(pretrained = False)
        num_final_in = model.fc.in_features
        model.fc = nn.Linear(num_final_in, 300)
        
        # Now that the architecture is defined same as above, let's load the model we would have trained above. 
        checkpoint = torch.load(MODEL_PATH)
        model.load_state_dict(checkpoint)
        
        
        # Let's freeze the same as above. Same code as above without the print statements
        child_counter = 0
        for child in model.children():
            if child_counter < 6:
                for param in child.parameters():
                    param.requires_grad = False
            elif child_counter == 6:
                children_of_child_counter = 0
                for children_of_child in child.children():
                    if children_of_child_counter < 1:
                        for param in children_of_child.parameters():
                            param.requires_grad = False
                    else:
                    children_of_child_counter += 1

            else:
                print("child ",child_counter," was not frozen")
            child_counter += 1
        
        # Now, let's define new layers that we want to add on top. 
        # Basically, these are just objects we define here. The "adding on top" is defined by the forward()
        # function which decides the flow of the input data into the model.
        
        # NOTE - Even the above model needs to be passed to self.
        self.vismodel = nn.Sequential(*list(model.children()))
        self.projective = nn.Linear(512,400)
        self.nonlinearity = nn.ReLU(inplace=True)
        self.projective2 = nn.Linear(400,300)
        
    
    # The forward function defines the flow of the input data and thus decides which layer/chunk goes on top of what.
    def forward(self,x):
        x = self.vismodel(x)
        x = torch.squeeze(x)
        x = self.projective(x)
        x = self.nonlinearity(x)
        x = self.projective2(x)
        return x

## SECTION 5 - CUSTOM LOSS FUNCTION

Now that we have our model all in place we can load anything and create any architecture we want. That leaves us with 2 important components in any pipeline - Loading the data, and the training part. Let's take a look at the training part. The two most important components of this step are the optimizer and the loss function. The loss function quantifies how far our existing model is from where we want to be, and the optimizer decides how to update parameters such that we can minimize the loss.

Sometimes, we need to define our own loss functions. And here are a few things to know about this -

- custom Loss functions are defined using a custom class too. They inherit from torch.nn.Module just like the custom model.
- Often, we need to change the dimenions of one of our inputs. This can be done using view() function.
- If we want to add a dimension to a tensor, use the unsqueeze() function.
- The value finally being returned by a loss function MUST BE a scalar value. Not a vector/tensor.
- The value being returned must be a Variable. This is so that it can be used to update the parameters. The best way to do so is to just make sure that both x and y being passed in are Variables. That way any function of the two will also be a Variable.
- A Pytorch Variable is just a Pytorch Tensor, but Pytorch is tracking the operations being done on it so that it can backpropagate to get the gradient.
Here I show a custom loss called Regress_Loss which takes as input 2 kinds of input x and y. Then it reshapes x to be similar to y and finally returns the loss by calculating L2 difference between reshaped x and y. This is a standard thing you'll run across very often in training networks.

Consider x to be shape (5,10) and y to be shape (5,5,10). So, we need to add a dimension to x, then repeat it along the added dimension to match the dimension of y. Then, (x-y) will be the shape (5,5,10). We will have to add over all three dimensions i.e. three torch.sum() to get a scalar.

In [ ]:
class Regress_Loss(torch.nn.Module):
    
    def __init__(self):
        super(Regress_Loss,self).__init__()
        
    def forward(self,x,y):
        y_shape = y.size()[1]
        x_added_dim = x.unsqueeze(1)
        x_stacked_along_dimension1 = x_added_dim.repeat(1,NUM_WORDS,1)
        diff = torch.sum((y - x_stacked_along_dimension1)**2,2)
        totloss = torch.sum(torch.sum(torch.sum(diff)))
        return totloss

link [here](https://spandan-madan.github.io/A-Collection-of-important-tasks-in-pytorch/)